In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import numpy as np
import keras
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input, MaxPooling2D,Conv2D,Dense, Dropout, Activation,Flatten, BatchNormalization
from keras.optimizers import Adam,sgd
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback, EarlyStopping
from keras.models import model_from_json
import h5py
import tensorflow as tf

In [17]:
os.listdir()
os.chdir('gdrive/My Drive/MyMIR')
os.listdir()
os.chdir('final_Xtrain')

FileNotFoundError: ignored

In [0]:
os.chdir('../final_Xtrain')

In [0]:
Xtrain = []
Ytrain = []
Xtest = []
Ytest = []
Xval = []
Yval = []

In [0]:
#Xtrain = np.load("data1.npy")
#Xtest = np.load("dataB.npy")
Xval = np.load("dataC.npy")
os.chdir('../final_Ytrain')
#Ytrain = np.load("Y1.npy")
#Ytest = np.load("YB.npy")
Yval = np.load("YC.npy")

In [0]:
#Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
#print(Xtrain.shape)
#Xtest = Xtest.reshape(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2],1)
#print(Xtest.shape)
Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
print(Xval.shape)

(1825, 128, 1255, 1)


In [0]:
def ac(y_true, y_pred):
  value, update_op = tf.metrics.auc(y_pred, y_true)
  return value

In [0]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [0]:
my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

In [0]:
model = Sequential()
model.add(Conv2D(32,(5,5),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32,(5,5),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(5,5),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) 
model.add(Dense(120,activation = 'relu'))
model.add(Dense(84,activation = 'relu'))
model.add(Dense(50,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',auc_roc])

In [0]:
model.summary()

ValueError: ignored

In [0]:
os.listdir()
os.chdir('../final_Xtrain')

In [22]:
for i in range(0,12):
  if i<10:
    Xtrain = np.load('data'+str(i)+'.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('Y'+str(i)+'.npy')
    os.chdir('../final_Xtrain')
  elif i == 10:
    Xtrain = np.load('dataA.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YA.npy')
    os.chdir('../final_Xtrain')
  else:
    Xtrain = np.load('dataB.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YB.npy')
    os.chdir('../final_Xtrain')
  Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
  #Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
  model.fit(Xtrain, Ytrain,batch_size=16,epochs=2,verbose=1,validation_split=0.01,shuffle=True)

Train on 1096 samples, validate on 12 samples
Epoch 1/2
1096/1096 [==============================] - 14s 13ms/step - loss: 0.4041 - acc: 0.9112 - auc_roc: 0.4943 - val_loss: 0.3077 - val_acc: 0.9100 - val_auc_roc: 0.5008
Epoch 2/2
1096/1096 [==============================] - 13s 12ms/step - loss: 0.1867 - acc: 0.9474 - auc_roc: 0.5009 - val_loss: 0.3098 - val_acc: 0.9100 - val_auc_roc: 0.5013
Train on 1527 samples, validate on 16 samples
Epoch 1/2
1527/1527 [==============================] - 16s 11ms/step - loss: 0.1902 - acc: 0.9482 - auc_roc: 0.5013 - val_loss: 0.2386 - val_acc: 0.9350 - val_auc_roc: 0.5013
Epoch 2/2
1527/1527 [==============================] - 16s 11ms/step - loss: 0.1819 - acc: 0.9482 - auc_roc: 0.5011 - val_loss: 0.2415 - val_acc: 0.9350 - val_auc_roc: 0.5011
Train on 1546 samples, validate on 16 samples
Epoch 1/2
1546/1546 [==============================] - 17s 11ms/step - loss: 0.1941 - acc: 0.9471 - auc_roc: 0.5011 - val_loss: 0.2435 - val_acc: 0.9312 - val_auc

In [30]:
tempx = [['dataD.npy','YD.npy'],['dataE.npy','YE.npy'],['dataF.npy','YF.npy']]
for tple in tempx:
  Xtestp,Ytestp = tple
  os.chdir('../final_Xtrain')
  Xtest = np.load(Xtestp)
  os.chdir('../final_Ytrain')
  Ytest = np.load(Ytestp)
  Xtest = Xtest.reshape(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2],1)
  predicted = model.predict(Xtest,batch_size=32)
  print(roc_auc_score(Ytest, predicted))
  with tf.Session() as sess:
    yt = tf.placeholder(tf.placeholder("float",[None,50]))
  print((tf.metrics.auc(labels = Ytest,predictions = predicted)))

SyntaxError: ignored

In [0]:
(os.chdir('/content/gdrive/My Drive/MyMIR/model')

In [0]:
# load json and create model
json_file = open('modelv1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelv1.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
model_json = model.to_json()
with open("modelv4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelv4.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
#os.chdir('/content/gdrive/My Drive/MyMIR')
os.listdir()
#model.save('/content/gdrive/My Drive/MyMIR/model/modelv1.h5')

['modelv1.json',
 'modelv1.h5',
 'modelv2.json',
 'modelv2.h5',
 'modelv3.json',
 'modelv3.h5',
 'modelv4.json',
 'modelv4.h5']